In [2]:
import pandas as pd
import folium
import json

In [3]:
df =pd.read_csv('./situation.csv',encoding='utf-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361 entries, 0 to 360
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   city      361 non-null    object
 1   province  361 non-null    object
 2   grade     361 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 8.6+ KB


In [4]:
df.head()

,city,province,grade
0,黔南州,贵州省,12
1,玉树州,青海省,14
2,永州市,湖南省,15
3,南平市,福建省,17
4,遵义市,贵州省,18


In [5]:
coord = pd.read_csv('./china_coordinates.csv',                    
     names=['postcode', 'city', 'lon', 'lat'])
coord.head()

,postcode,city,lon,lat
0,110000,北京市,116.395645,39.929986
1,110101,东城区,113.612838,37.857865
2,110102,西城区,113.612838,37.857865
3,110105,朝阳区,116.521695,39.958953
4,110106,丰台区,116.258370,39.841938


In [6]:
coord= pd.merge(coord,df,on="city")
coord

,postcode,city,lon,lat,province,grade
0,110000,北京市,116.395645,39.929986,北京市,67
1,120000,天津市,117.210813,39.143930,天津市,67
2,130100,石家庄市,114.522082,38.048958,河北省,87
3,130200,唐山市,118.183451,39.650531,河北省,62
4,130300,秦皇岛市,119.604368,39.945462,河北省,74
...,...,...,...,...,...,...
320,653100,喀什地区,75.992973,39.470627,新疆维吾尔自治区,74
321,654200,塔城地区,82.974881,46.758684,新疆维吾尔自治区,59
322,654300,阿勒泰地区,88.137915,47.839744,新疆维吾尔自治区,26
323,659001,石河子市,86.041865,44.308259,新疆维吾尔自治区,44


In [7]:
grade = coord.groupby('province').grade.agg('max')
grade = pd.DataFrame(grade).reset_index()
grade

,province,grade
0,上海市,34
1,云南省,102
2,内蒙古自治区,67
3,北京市,67
4,吉林省,93
5,四川省,69
6,天津市,67
7,宁夏回族自治区,73
8,安徽省,51
9,山东省,67


In [8]:
grade.head()

,province,grade
0,上海市,34
1,云南省,102
2,内蒙古自治区,67
3,北京市,67
4,吉林省,93


In [16]:
type(coord['lon'][0])

numpy.float64

In [12]:
for d in coord.itertuples():
    print(d)

Pandas(Index=0, postcode=110000, city='北京市', lon=116.39564503787868, lat=39.929985778080244, province='北京市', grade=67)
Pandas(Index=1, postcode=120000, city='天津市', lon=117.21081309155257, lat=39.14392990331008, province='天津市', grade=67)
Pandas(Index=2, postcode=130100, city='石家庄市', lon=114.52208184420799, lat=38.048958314615504, province='河北省', grade=87)
Pandas(Index=3, postcode=130200, city='唐山市', lon=118.183450597734, lat=39.6505309225366, province='河北省', grade=62)
Pandas(Index=4, postcode=130300, city='秦皇岛市', lon=119.604367616118, lat=39.9454615658976, province='河北省', grade=74)
Pandas(Index=5, postcode=130400, city='邯郸市', lon=114.482693932342, lat=36.6093079284712, province='河北省', grade=66)
Pandas(Index=6, postcode=130500, city='邢台市', lon=114.52048681294399, lat=37.069531196911996, province='河北省', grade=71)
Pandas(Index=7, postcode=130600, city='保定市', lon=115.494810169076, lat=38.886564548027394, province='河北省', grade=69)
Pandas(Index=8, postcode=130700, city='张家口市', lon=114.8937815

In [7]:
m = folium.Map(location=[39.929986, 116.395645], zoom_start=4)
geo_data = json.load(open('./100000_full.json',encoding='utf-8'))
folium.Choropleth(
    geo_data= geo_data,
    data=grade.set_index(grade.province),
    columns=['province', 'grade'],
    key_on='feature.properties.name',
    fill_color='Reds',
    bins = [20, 30, 50, 70, 90, 110]
).add_to(m)
for d in coord.itertuples():
    folium.Marker(
        location=[d.lat, d.lon],
        icon=folium.Icon(color='red', prefix='fa', icon='warning') if d.grade> 90 else
        (folium.Icon(color='orange') if d.grade >50
          else folium.Icon(color='blue', prefix='fa', icon='thumbs-up')),
        tooltip=" {} 's AQI is {} ".format(d.city, int(d.grade))
    ).add_to(m)
m

In [8]:
coord

,postcode,city,lon,lat,province,grade
0,110000,北京市,116.395645,39.929986,北京市,67
1,120000,天津市,117.210813,39.143930,天津市,67
2,130100,石家庄市,114.522082,38.048958,河北省,87
3,130200,唐山市,118.183451,39.650531,河北省,62
4,130300,秦皇岛市,119.604368,39.945462,河北省,74
...,...,...,...,...,...,...
320,653100,喀什地区,75.992973,39.470627,新疆维吾尔自治区,74
321,654200,塔城地区,82.974881,46.758684,新疆维吾尔自治区,59
322,654300,阿勒泰地区,88.137915,47.839744,新疆维吾尔自治区,26
323,659001,石河子市,86.041865,44.308259,新疆维吾尔自治区,44


In [9]:
hb = coord.loc[coord['province'] == '河北省',['city','grade']]
hb

,city,grade
2,石家庄市,87
3,唐山市,62
4,秦皇岛市,74
5,邯郸市,66
6,邢台市,71
7,保定市,69
8,张家口市,53
9,承德市,54
10,沧州市,56
11,廊坊市,63


In [10]:
import vincent
bar_chart = vincent.Bar(hb, iter_idx="city", width=600, height=300)
bar_json = bar_chart.to_json()
bar_dict = json.loads(bar_json)
m1 = folium.Map(location=[39.929986, 116.395645], zoom_start=4)
geo_data = json.load(open('./100000_full.json',encoding='utf-8'))
folium.Choropleth(
    geo_data= geo_data,
    data=grade.set_index(grade.province),
    columns=['province', 'grade'],
    key_on='feature.properties.name',
    fill_color='Reds',
    bins = [20, 30, 50, 70, 90, 110]
).add_to(m1)
popup = folium.Popup()
folium.Vega(bar_chart, height=350, width=650).add_to(popup)
folium.Marker([38.048958, 114.522082], popup=popup,
              tooltip='Click here to see the AQI comparison of cities in Hebei Province',
              icon=folium.Icon(color='cadetblue', prefix='fa', icon='bar-chart')).add_to(m1)
m1

In [11]:
from folium.plugins import HeatMap
heat_map = folium.Map(location=[39.929986, 116.395645], zoom_start=4)
heatdata = coord[['lat','lon','grade']].values.tolist()
HeatMap(heatdata).add_to(heat_map)
heat_map